In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install numpy scikit-image scipy


In [ ]:
import numpy as np
from skimage import io, img_as_float
from skimage.restoration import richardson_lucy, wiener, unsupervised_wiener, denoise_tv_chambolle
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from scipy.signal import convolve2d

# Load and preprocess the fluorescence image
image = img_as_float(io.imread('fluorescence_image.tif'))
psf = np.ones((5, 5)) / 25  # Example Point Spread Function (PSF)

# Function to calculate SNR
def calculate_snr(original, restored):
    noise = original - restored
    signal_power = np.mean(original ** 2)
    noise_power = np.mean(noise ** 2)
    return 10 * np.log10(signal_power / noise_power)

# Blind Deconvolution (using Richardson-Lucy with TV as a proxy)
def blind_deconvolution(image, psf, iterations=30):
    estimated_psf = np.copy(psf)
    deconvolved_image = np.copy(image)
    for _ in range(iterations):
        deconvolved_image = richardson_lucy(deconvolved_image, estimated_psf, iterations=10)
        estimated_psf = richardson_lucy(convolve2d(deconvolved_image, psf, mode='same'), image, iterations=10)
    return deconvolved_image

# Richardson-Lucy with Total Variation (RLD-TV)
def richardson_lucy_tv(image, psf, iterations=30, regularization_weight=0.1):
    deconvolved_image = richardson_lucy(image, psf, iterations=iterations)
    deconvolved_image_tv = denoise_tv_chambolle(deconvolved_image, weight=regularization_weight)
    return deconvolved_image_tv

# Apply Deconvolution Algorithms
rl_deconv = richardson_lucy(image, psf, iterations=30)
wiener_deconv = wiener(image, psf, balance=0.1)
unsupervised_deconv, _ = unsupervised_wiener(image, psf)
blind_deconv = blind_deconvolution(image, psf, iterations=30)
rld_tv_deconv = richardson_lucy_tv(image, psf, iterations=30, regularization_weight=0.1)

# Performance Metrics
metrics = {
    'Algorithm': [],
    'SNR': [],
    'PSNR': [],
    'SSIM': []
}

def evaluate_performance(algorithm_name, deconvolved_image):
    snr_value = calculate_snr(image, deconvolved_image)
    psnr_value = psnr(image, deconvolved_image)
    ssim_value = ssim(image, deconvolved_image)

    metrics['Algorithm'].append(algorithm_name)
    metrics['SNR'].append(snr_value)
    metrics['PSNR'].append(psnr_value)
    metrics['SSIM'].append(ssim_value)

evaluate_performance('Richardson-Lucy', rl_deconv)
evaluate_performance('Wiener', wiener_deconv)
evaluate_performance('Unsupervised Wiener', unsupervised_deconv)
evaluate_performance('Blind Deconvolution', blind_deconv)
evaluate_performance('RLD with TV', rld_tv_deconv)

# Display the results
for i in range(len(metrics['Algorithm'])):
    print(f"Algorithm: {metrics['Algorithm'][i]}")
    print(f"  SNR:  {metrics['SNR'][i]:.2f} dB")
    print(f"  PSNR: {metrics['PSNR'][i]:.2f} dB")
    print(f"  SSIM: {metrics['SSIM'][i]:.4f}")
    print()

# Optional: Save the deconvolved images
io.imsave('richardson_lucy_deconv.tif', rl_deconv)
io.imsave('wiener_deconv.tif', wiener_deconv)
io.imsave('unsupervised_wiener_deconv.tif', unsupervised_deconv)
io.imsave('blind_deconv.tif', blind_deconv)
io.imsave('rld_tv_deconv.tif', rld_tv_deconv)
